In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
import numpy as np
import osmnx as ox
import random
from math import radians, cos, sqrt
from sklearn.model_selection import train_test_split

In [1]:
# Hàm tính khoảng cách Euclidean
def euclidean(lat1, lon1, lat2, lon2):
    lat_mean = radians((lat1 + lat2) / 2)
    dx = (lon2 - lon1) * 111320 * cos(lat_mean)
    dy = (lat2 - lat1) * 111320
    return sqrt(dx**2 + dy**2)

In [3]:

G = ox.graph_from_xml('map_new.osm')

G_simple = G.to_undirected()

nodes = list(G_simple.nodes)

X = []
y = []

import pandas as pd
import numpy as np

df= pd.read_csv('shortest_path_dataset.csv')
X = df[['node1_x','node1_y','node2_x','node2_y']].values
y = df[['shortest_distance','nodes_between']].values

# Compute Euclidean distances for all rows
euclidean_distances = np.array([euclidean(lat1, lon1, lat2, lon2) for lat1, lon1, lat2, lon2 in X])

# Concatenate as a new feature column
X = np.hstack([X, euclidean_distances.reshape(-1, 1)])

X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [26]:
class HeuristicNet(nn.Module):
    def __init__(self):
        super(HeuristicNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.net(x)

model = HeuristicNet()
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

In [27]:

for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
        
        # Thêm đánh giá trên tập validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(torch.tensor(X_test, dtype=torch.float32))
            val_loss = criterion(val_outputs, torch.tensor(y_test, dtype=torch.float32))
            print(f"Validation Loss: {val_loss.item():.4f}")

Epoch 0, Loss: 644.6840
Validation Loss: 640.0905
Epoch 10, Loss: 608.2023
Validation Loss: 604.7014
Epoch 20, Loss: 574.2244
Validation Loss: 570.4298
Epoch 30, Loss: 534.6895
Validation Loss: 530.0490
Epoch 40, Loss: 482.0434
Validation Loss: 475.7006
Epoch 50, Loss: 409.7651
Validation Loss: 401.0832
Epoch 60, Loss: 311.6351
Validation Loss: 300.1382
Epoch 70, Loss: 212.2134
Validation Loss: 207.0844
Epoch 80, Loss: 198.3612
Validation Loss: 198.5630
Epoch 90, Loss: 199.5437
Validation Loss: 198.8273
Epoch 100, Loss: 193.7736
Validation Loss: 192.8310
Epoch 110, Loss: 189.4731
Validation Loss: 188.7518
Epoch 120, Loss: 186.9424
Validation Loss: 186.2736
Epoch 130, Loss: 184.8355
Validation Loss: 184.2051
Epoch 140, Loss: 183.0142
Validation Loss: 182.3943
Epoch 150, Loss: 181.3418
Validation Loss: 180.7442
Epoch 160, Loss: 179.8631
Validation Loss: 179.2915
Epoch 170, Loss: 178.5538
Validation Loss: 178.0105
Epoch 180, Loss: 177.3773
Validation Loss: 176.8543
Epoch 190, Loss: 176.31

In [28]:
def predict_distance(node1, node2, model):
    try:
        # Lấy tọa độ
        lat1 = G.nodes[node1]['y']
        lon1 = G.nodes[node1]['x']
        lat2 = G.nodes[node2]['y']
        lon2 = G.nodes[node2]['x']
        
        # Tính Euclidean distance
        euc_dist = euclidean(lat1, lon1, lat2, lon2)
        
        # Tạo tensor input đúng kích thước
        features = torch.tensor([lat1, lon1, lat2, lon2, euc_dist], 
                              dtype=torch.float32).reshape(1, -1)
        
        # Dự đoán
        model.eval()
        with torch.no_grad():
            output = model(features)
            
        return output[0][0].item()  # Lấy giá trị khoảng cách
        
    except Exception as e:
        print(f"Lỗi khi dự đoán: {str(e)}")
        return None

In [31]:
# Test model
nodes = list(G.nodes)[:10]
for i, node1 in enumerate(nodes):
    for j, node2 in enumerate(nodes):
        if i != j:
            pred_dist = predict_distance(node1, node2, model)
            if pred_dist is not None:
                print("\n")
                print(f"Khoảng cách từ {node1} đến {node2}: {pred_dist:.2f} mét")
                print(f"Khoảng cách thực tế theo đường chim bay: {euclidean(G.nodes[node1]['y'], G.nodes[node1]['x'], G.nodes[node2]['y'], G.nodes[node2]['x']):.2f} mét")
                print(f"Khoảng cách thực tế từ shortest_path: {nx.shortest_path_length(G, source=node1, target=node2, weight='length'):.2f} mét")



Khoảng cách từ 5704540253 đến 5704540254: 702.77 mét
Khoảng cách thực tế theo đường chim bay: 255.33 mét
Khoảng cách thực tế từ shortest_path: 570.00 mét


Khoảng cách từ 5704540253 đến 5704540256: 835.71 mét
Khoảng cách thực tế theo đường chim bay: 329.87 mét
Khoảng cách thực tế từ shortest_path: 482.14 mét


Khoảng cách từ 5704540253 đến 6441123934: 1411.68 mét
Khoảng cách thực tế theo đường chim bay: 979.22 mét
Khoảng cách thực tế từ shortest_path: 1329.11 mét


Khoảng cách từ 5704540253 đến 6688497762: 2466.99 mét
Khoảng cách thực tế theo đường chim bay: 1967.19 mét
Khoảng cách thực tế từ shortest_path: 2183.92 mét


Khoảng cách từ 5704540253 đến 10826334340: 451.56 mét
Khoảng cách thực tế theo đường chim bay: 161.14 mét
Khoảng cách thực tế từ shortest_path: 816.84 mét


Khoảng cách từ 5704540253 đến 12223897784: 302.67 mét
Khoảng cách thực tế theo đường chim bay: 114.32 mét
Khoảng cách thực tế từ shortest_path: 146.05 mét


Khoảng cách từ 5704540253 đến 5706891493: 1169.20 mét
K

In [30]:
import joblib
joblib.dump(model, 'ml_heuristic_model.pkl')

['ml_heuristic_model.pkl']